Primero, obtengo datos sobre precios históricos de criptomonedas (como Bitcoin, Ethereum, Litecoin, etc.) y variables adicionales relacionadas, como volumen de operaciones, capitalización de mercado y métricas de redes sociales. Para esto utilizo la API de CoinGecko.
Luego envuelvo todo en una funcion lo convierto en un DataFrame y le agrego las columnas relevantes, que van a ser precio, capitalizacion del mercado y volumen.

Problema comercial y analítico: predecir la dirección futura de los precios de las criptomonedas, identificar oportunidades de inversión y gestionar el riesgo en una cartera de criptoactivos.

## Funcion para obtener datos de la API y concatenar los activos en un unico dataframe

In [1]:
import pandas as pd
import requests
from datetime import datetime

# Función para obtener datos históricos de CoinGecko
def get_historical_data(coin_id, vs_currency='usd', days='max'):
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart?vs_currency={vs_currency}&days={days}&interval=daily'
    response = requests.get(url)
    data = response.json()

    prices = data['prices']
    market_caps = data['market_caps']
    total_volumes = data['total_volumes']

    # Convertir los datos en un DataFrame de pandas
    df = pd.DataFrame(prices, columns=['timestamp', f'{coin_id}_price'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df[f'{coin_id}_market_cap'] = [mc[1] for mc in market_caps]
    df[f'{coin_id}_total_volume'] = [tv[1] for tv in total_volumes]

    return df

# Lista de criptomonedas para obtener datos
cryptos = ['bitcoin', 'ethereum', 'litecoin', 'ripple', 'cardano']

# Obtener datos históricos para cada criptomoneda y combinarlos en un DataFrame
crypto_data = None
for crypto in cryptos:
    temp_data = get_historical_data(crypto)
    if crypto_data is None:
        crypto_data = temp_data
    else:
        crypto_data = crypto_data.join(temp_data)

# Guardar el DataFrame en un archivo CSV
crypto_data.to_csv('crypto_historical_data.csv')


In [4]:
print(crypto_data)

                     bitcoin_price  bitcoin_market_cap  bitcoin_total_volume  \
timestamp                                                                      
2013-04-28 00:00:00     135.300000        1.500518e+09          0.000000e+00   
2013-04-29 00:00:00     141.960000        1.575032e+09          0.000000e+00   
2013-04-30 00:00:00     135.300000        1.501657e+09          0.000000e+00   
2013-05-01 00:00:00     117.000000        1.298952e+09          0.000000e+00   
2013-05-02 00:00:00     103.430000        1.148668e+09          0.000000e+00   
...                            ...                 ...                   ...   
2023-04-29 00:00:00   29339.994998        5.681539e+11          1.873080e+10   
2023-04-30 00:00:00   29217.944047        5.657144e+11          9.711415e+09   
2023-05-01 00:00:00   29362.056214        5.685135e+11          1.470701e+10   
2023-05-02 00:00:00   28125.501156        5.444217e+11          1.998017e+10   
2023-05-02 18:31:01   28615.175335      

## Familiarizando el Dataframe

In [5]:
# Imprimir las columnas del DataFrame
print("Columnas del DataFrame:")
print(crypto_data.columns)

Columnas del DataFrame:
Index(['bitcoin_price', 'bitcoin_market_cap', 'bitcoin_total_volume',
       'ethereum_price', 'ethereum_market_cap', 'ethereum_total_volume',
       'litecoin_price', 'litecoin_market_cap', 'litecoin_total_volume',
       'ripple_price', 'ripple_market_cap', 'ripple_total_volume',
       'cardano_price', 'cardano_market_cap', 'cardano_total_volume'],
      dtype='object')


In [6]:
# Imprimir los primeros 10 datos del DataFrame
print("\nPrimeros 10 datos:")
print(crypto_data.head(10))


Primeros 10 datos:
            bitcoin_price  bitcoin_market_cap  bitcoin_total_volume  \
timestamp                                                             
2013-04-28         135.30        1.500518e+09                   0.0   
2013-04-29         141.96        1.575032e+09                   0.0   
2013-04-30         135.30        1.501657e+09                   0.0   
2013-05-01         117.00        1.298952e+09                   0.0   
2013-05-02         103.43        1.148668e+09                   0.0   
2013-05-03          91.01        1.011066e+09                   0.0   
2013-05-04         111.25        1.236352e+09                   0.0   
2013-05-05         116.79        1.298378e+09                   0.0   
2013-05-06         118.33        1.315992e+09                   0.0   
2013-05-07         106.40        1.183766e+09                   0.0   

            ethereum_price  ethereum_market_cap  ethereum_total_volume  \
timestamp                                            

In [7]:
# Imprimir los últimos 10 datos del DataFrame
print("\nÚltimos 10 datos:")
print(crypto_data.tail(10))


Últimos 10 datos:
                     bitcoin_price  bitcoin_market_cap  bitcoin_total_volume  \
timestamp                                                                      
2023-04-24 00:00:00   27606.578348        5.337532e+11          1.143565e+10   
2023-04-25 00:00:00   27511.635682        5.322908e+11          1.644310e+10   
2023-04-26 00:00:00   28351.218248        5.483597e+11          1.706182e+10   
2023-04-27 00:00:00   28352.191321        5.486387e+11          3.054003e+10   
2023-04-28 00:00:00   29483.521705        5.702580e+11          2.623860e+10   
2023-04-29 00:00:00   29339.994998        5.681539e+11          1.873080e+10   
2023-04-30 00:00:00   29217.944047        5.657144e+11          9.711415e+09   
2023-05-01 00:00:00   29362.056214        5.685135e+11          1.470701e+10   
2023-05-02 00:00:00   28125.501156        5.444217e+11          1.998017e+10   
2023-05-02 18:31:01   28615.175335        5.544156e+11          1.869104e+10   

                    

Variables interesantes: precio de cierre, volumen de operaciones, capitalización de mercado, número de transacciones, sentimiento en redes sociales, volatilidad y correlación entre criptomonedas.

## Datos interesantes: Precios de cierre, volumen, volatilidad y matriz de correlacion entre los activos Bitcoin y Ethereum.

In [17]:
#Para ver precios de cierre de alguna cripto:
coin_ids = ['bitcoin', 'ethereum']

for coin_id in coin_ids:
    print(f"Precios de cierre de {coin_id}:")
    print(crypto_data[f'{coin_id}_price'])
    print("\n")

Precios de cierre de bitcoin:
timestamp
2013-04-28 00:00:00      135.300000
2013-04-29 00:00:00      141.960000
2013-04-30 00:00:00      135.300000
2013-05-01 00:00:00      117.000000
2013-05-02 00:00:00      103.430000
                           ...     
2023-04-29 00:00:00    29339.994998
2023-04-30 00:00:00    29217.944047
2023-05-01 00:00:00    29362.056214
2023-05-02 00:00:00    28125.501156
2023-05-02 18:31:01    28615.175335
Name: bitcoin_price, Length: 3656, dtype: float64


Precios de cierre de ethereum:
timestamp
2013-04-28 00:00:00            NaN
2013-04-29 00:00:00            NaN
2013-04-30 00:00:00            NaN
2013-05-01 00:00:00            NaN
2013-05-02 00:00:00            NaN
                          ...     
2023-04-29 00:00:00    1894.428084
2023-04-30 00:00:00    1905.273852
2023-05-01 00:00:00    1885.391935
2023-05-02 00:00:00    1832.725799
2023-05-02 18:31:01            NaN
Name: ethereum_price, Length: 3656, dtype: float64




In [18]:
# Volumen total:

for coin_id in coin_ids:
    print(f"Volumen total de {coin_id}:")
    print(crypto_data[f'{coin_id}_total_volume'])
    print("\n")


Volumen total de bitcoin:
timestamp
2013-04-28 00:00:00    0.000000e+00
2013-04-29 00:00:00    0.000000e+00
2013-04-30 00:00:00    0.000000e+00
2013-05-01 00:00:00    0.000000e+00
2013-05-02 00:00:00    0.000000e+00
                           ...     
2023-04-29 00:00:00    1.873080e+10
2023-04-30 00:00:00    9.711415e+09
2023-05-01 00:00:00    1.470701e+10
2023-05-02 00:00:00    1.998017e+10
2023-05-02 18:31:01    1.869104e+10
Name: bitcoin_total_volume, Length: 3656, dtype: float64


Volumen total de ethereum:
timestamp
2013-04-28 00:00:00             NaN
2013-04-29 00:00:00             NaN
2013-04-30 00:00:00             NaN
2013-05-01 00:00:00             NaN
2013-05-02 00:00:00             NaN
                           ...     
2023-04-29 00:00:00    8.853392e+09
2023-04-30 00:00:00    5.262010e+09
2023-05-01 00:00:00    7.328083e+09
2023-05-02 00:00:00    9.673378e+09
2023-05-02 18:31:01             NaN
Name: ethereum_total_volume, Length: 3656, dtype: float64




In [19]:
# Calcular rendimientos diarios
returns_data = crypto_data[[f'{coin_id}_price' for coin_id in coin_ids]].pct_change()

# Calcular volatilidad (desviación estándar de los rendimientos)
volatility = returns_data.std()

print("Volatilidad:")
print(volatility)
print("\n")

Volatilidad:
bitcoin_price     0.040345
ethereum_price    0.060081
dtype: float64




In [20]:
# Calcular matriz de correlacion

matriz_correlacion = returns_data.corr()

print("Matriz de correlación:")
print(matriz_correlacion)
print("\n")


Matriz de correlación:
                bitcoin_price  ethereum_price
bitcoin_price        1.000000        0.558112
ethereum_price       0.558112        1.000000


